In [ ]:
%load_ext autoreload
%autoreload 2

from soccerai.data.data import create_dataset

df = create_dataset("prova.parquet")

In [ ]:
import polars as pl

df = pl.read_parquet("prova.parquet")
print(df["possessionEventType"].value_counts())

In [ ]:
df

In [ ]:
df.filter(pl.col("jerseyNum").is_null())

* trasformare `playerName_right` e `playerName`, se == allora -> valore booleano 1 che indica il possesso della palla, altrimenti 0. Droppare successivamente `playerName_right`, `playerName` `Full Name`.
* `teamName` -> OneHotEncoding?
* `playerRole` -> trasformazione?
* bisogna assolutamente togliere i frames dello shot dalle catene positive
* caricare dataset -> preprocessing banale -> trasformazione in grafo -> training GCN?
* `possessionEventType` lo userei per taggare gli edge
* quanto dura un evento è importante? (`startTime`, `endTime`, `duration`)
* `frameTime` ci indicherebbe a che punto della partita siamo!
* sviluppo futuro: aggiungere informazioni della squadra precedenti negli ultimi 4 anni
* fare brainstorming sulla palla, il nodo palla
* grafo spesso omogeneo, nodi -> giocatori, archi interazioni on ball
* il team che ha possesso palla (playerName che abbiamo dall'evento e il suo team corrisponde al team attaccante, altrimenti sarà non attaccante)
* il grafo è omogeno in counterattacks

In [ ]:
print(df.height)

In [ ]:
df.null_count()